In [51]:
import pandas as pd
import datetime
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
import xgboost as xgb

In [2]:
data = pd.read_csv('holdout_set.csv', encoding = "ISO-8859-1")

In [3]:
data.head()

,Engagements,Followers at Posting,Created,Type,Description
0,NaN,36984682,2019-05-22 00:31:55 EDT,Photo,The @raptors even up the Eastern Conference Fi...
1,NaN,36984682,2019-05-21 23:20:41 EDT,Photo,The @raptors even the Eastern Conference Final...
2,NaN,36984682,2019-05-21 21:18:51 EDT,Video,@sergeibaka keeps it alive for the @raptors! (...
3,NaN,36955156,2019-05-21 18:20:09 EDT,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...
4,NaN,36955156,2019-05-21 15:04:31 EDT,Video,?? the @raptors & @bucks in transition! #NBABr...


In [4]:
data['Description']= data.Description.fillna('')

In [5]:
data['Len_Desc'] = data['Description'].str.len()

In [6]:
data['Tagged_Count'] = data['Description'].str.count('@')

In [7]:
data.isna().sum() 

Engagements             1000
Followers at Posting       0
Created                    0
Type                       0
Description                0
Len_Desc                   0
Tagged_Count               0
dtype: int64

In [8]:
data['DayOfWeek']= pd.to_datetime(data['Created']).dt.day_name()

In [9]:
data['Created']= pd.to_datetime(data['Created'])

In [10]:
def playoffDate(date): 
    if date > pd.Timestamp('2019-4-13'): 
        return 1 
    else: 
        return 0

In [11]:
data['playOffDate']= data.Created.map(lambda a: playoffDate(a.tz_localize(None)))

In [12]:
data['month'] = data.Created.map(lambda a: a.month)

In [13]:
data['hour'] = data.Created.map(lambda a: a.hour)

In [14]:
def timeOfDay(hour): 
    if hour < 6: 
        return 'Night' 
    elif hour <12: 
        return 'Morning'
    elif hour <18: 
        return 'Afternoon'
    else:
        return'Evening'

In [15]:
data['timeOfDay']= data.hour.map(lambda a: timeOfDay(a))

In [16]:
def proportionOfTarget(caption): 
    #In this case target will be all non normal text
    targetCount = 0
    if not caption: return 0
    lst = caption.split(' ')
    count = len(lst)
    if count == 0: return 0
    for word in lst: 
        if len(word) ==0: continue 
        if word[0] in ['@', '#']: targetCount += 1
        elif '??' in word: targetCount+= 1
    return float(targetCount)/float(count)
    
    

In [17]:
data['propOfSpecialTexts'] = data.Description.map(lambda a: proportionOfTarget(a))

In [18]:
data.head()

,Engagements,Followers at Posting,Created,Type,Description,Len_Desc,Tagged_Count,DayOfWeek,playOffDate,month,hour,timeOfDay,propOfSpecialTexts
0,NaN,36984682,2019-05-22 04:31:55,Photo,The @raptors even up the Eastern Conference Fi...,55,1,Wednesday,1,5,4,Night,0.111111
1,NaN,36984682,2019-05-22 03:20:41,Photo,The @raptors even the Eastern Conference Final...,92,2,Wednesday,1,5,3,Night,0.133333
2,NaN,36984682,2019-05-22 01:18:51,Video,@sergeibaka keeps it alive for the @raptors! (...,60,3,Wednesday,1,5,1,Night,0.444444
3,NaN,36955156,2019-05-21 22:20:09,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,105,3,Tuesday,1,5,22,Evening,0.375000
4,NaN,36955156,2019-05-21 19:04:31,Video,?? the @raptors & @bucks in transition! #NBABr...,121,3,Tuesday,1,5,19,Evening,0.375000


In [19]:
data['shoutouts']= data.Description.map(lambda x: re.findall('@\w*', x))
allVals = {}
for listItem in data['shoutouts']:
    for item in listItem:
        if item.lower() not in allVals: allVals[item.lower()] = 0
        allVals[item.lower()] = allVals[item.lower()] + 1
allVals

{'@raptors': 30,
 '@bucks': 18,
 '@sergeibaka': 3,
 '@nbaontnt': 60,
 '@warriors': 55,
 '@youtube': 3,
 '@stephencurry30': 29,
 '@money23green': 8,
 '@trailblazers': 16,
 '@nbaonespn': 17,
 '@qcook323': 1,
 '@meyersleonard11': 1,
 '@3jmccollum': 2,
 '@jharden13': 29,
 '@houstonrockets': 33,
 '@cp3': 12,
 '@ccapela15': 7,
 '@jimmybutler': 12,
 '@bensimmons': 17,
 '@sixers': 39,
 '@kyle_lowry7': 11,
 '@biggame_08': 2,
 '@nuggets': 18,
 '@celtics': 28,
 '@_kw15': 6,
 '@hornets': 14,
 '@jmglitxh27': 5,
 '@damianlillard': 7,
 '@easymoneysniper': 10,
 '@klaythompson': 6,
 '@budweiserusa': 2,
 '@fchwpo': 5,
 '@theofficialai3': 3,
 '@willthethrillb5': 1,
 '@trezz24': 2,
 '@andre': 3,
 '@nbahistory': 14,
 '@spurs': 12,
 '@nbatv': 25,
 '@utahjazz': 12,
 '@spidadmitchell': 16,
 '@giannis_an34': 17,
 '@detroitpistons': 11,
 '@laclippers': 14,
 '@kyrieirving': 16,
 '@pacers': 6,
 '@brooklynnets': 11,
 '@okcthunder': 26,
 '@djaugustin': 1,
 '@orlandomagic': 7,
 '@dwyanewade': 24,
 '@swish41': 8,
 '@

In [20]:
sorted([(vals,key) for key,vals in allVals.items()], reverse = True)

[(60, '@nbaontnt'),
 (60, '@kingjames'),
 (55, '@warriors'),
 (39, '@sixers'),
 (37, '@lakers'),
 (35, '@cavs'),
 (33, '@houstonrockets'),
 (30, '@raptors'),
 (29, '@stephencurry30'),
 (29, '@jharden13'),
 (28, '@celtics'),
 (26, '@okcthunder'),
 (25, '@russwest44'),
 (25, '@nbatv'),
 (24, '@dwyanewade'),
 (18, '@nuggets'),
 (18, '@bucks'),
 (17, '@washwizards'),
 (17, '@nbaonespn'),
 (17, '@giannis_an34'),
 (17, '@bensimmons'),
 (16, '@trailblazers'),
 (16, '@spidadmitchell'),
 (16, '@kyrieirving'),
 (15, '@nbaallstar'),
 (15, '@miamiheat'),
 (14, '@nbahistory'),
 (14, '@laclippers'),
 (14, '@hornets'),
 (13, '@traeyoung'),
 (13, '@dallasmavs'),
 (12, '@zo'),
 (12, '@ygtrece'),
 (12, '@utahjazz'),
 (12, '@spurs'),
 (12, '@nyknicks'),
 (12, '@kevindurant'),
 (12, '@joelembiid'),
 (12, '@jimmybutler'),
 (12, '@cp3'),
 (11, '@kyle_lowry7'),
 (11, '@detroitpistons'),
 (11, '@brooklynnets'),
 (10, '@timberwolves'),
 (10, '@easymoneysniper'),
 (10, '@desmith4'),
 (8, '@swish41'),
 (8, '@pel

In [24]:
teams = ['@nuggets ','@spurs ', '@okcthunder ', '@pelicansnba ','@dallasmavs ', '@trailblazers ', '@utahjazz ',
        '@bucks ', '@sixers ','@raptors ','@laclippers ', '@miamiheat ', '@pacers ', '@houstonrockets ',
        '@timberwolves ', '@cavs ', '@warriors ','@celtics ','@lakers ','@sacramentokings ', '@orlandomagic ', '@detroitpistons ',
         '@brooklynnets ','@atlhawks ', '@memgrizz ', '@suns ', '@hornets ', '@washwizards ', '@nyknicks ', '@chicagobulls '
        ]

In [25]:
def teamTagged(lst): 
    for tag in lst: 
        if tag in teams: 
            return 1 
    return 0 

In [26]:
data['teamTagged']= data.shoutouts.map(lambda a: teamTagged(a))

In [27]:
def otherTagged(lst): 
    for tag in lst: 
        if tag not in teams: 
            return 1 
    return 0 

data['otherTagged']= data.shoutouts.map(lambda a: otherTagged(a))

In [28]:
top3 = ['@kyrieirving ','@kingjames ', '@stephencurry30 ']
bottom3 = ['@nuggets ','@spurs ','@okcthunder ' ]

def top3Tagged(lst): 
    for tag in lst: 
        if tag in top3: 
            return 1 
    return 0 

data['top3Tagged']= data.shoutouts.map(lambda a: top3Tagged(a))

def bottom3Tagged(lst): 
    for tag in lst: 
        if tag in bottom3: 
            return 1 
    return 0 

data['bottom3Tagged']= data.shoutouts.map(lambda a: bottom3Tagged(a))

In [29]:
data['hashtags']= data.Description.map(lambda x: re.findall('#\w*', x))


In [30]:
data['hasHashtag']=data.hashtags.map(lambda x: x != [])

In [33]:
data['monthDateYear']= data.Created.map(lambda x: x.strftime('%Y-%m-%d'))

In [34]:
data.head()

,Engagements,Followers at Posting,Created,Type,Description,Len_Desc,Tagged_Count,DayOfWeek,playOffDate,month,...,timeOfDay,propOfSpecialTexts,shoutouts,teamTagged,otherTagged,top3Tagged,bottom3Tagged,hashtags,hasHashtag,monthDateYear
0,NaN,36984682,2019-05-22 04:31:55,Photo,The @raptors even up the Eastern Conference Fi...,55,1,Wednesday,1,5,...,Night,0.111111,[@raptors],0,1,0,0,[],False,2019-05-22
1,NaN,36984682,2019-05-22 03:20:41,Photo,The @raptors even the Eastern Conference Final...,92,2,Wednesday,1,5,...,Night,0.133333,"[@raptors, @bucks]",0,1,0,0,[],False,2019-05-22
2,NaN,36984682,2019-05-22 01:18:51,Video,@sergeibaka keeps it alive for the @raptors! (...,60,3,Wednesday,1,5,...,Night,0.444444,"[@sergeibaka, @raptors, @nbaontnt]",0,1,0,0,[],False,2019-05-22
3,NaN,36955156,2019-05-21 22:20:09,Video,???? The SUPERHUMAN HANDS of Giannis & Kawhi ?...,105,3,Tuesday,1,5,...,Evening,0.375000,"[@bucks, @raptors, @nbaontnt]",0,1,0,0,[],False,2019-05-21
4,NaN,36955156,2019-05-21 19:04:31,Video,?? the @raptors & @bucks in transition! #NBABr...,121,3,Tuesday,1,5,...,Evening,0.375000,"[@raptors, @bucks, @nbaontnt]",0,1,0,0,[#NBABreakdown],True,2019-05-21


In [35]:
x =data.groupby(['monthDateYear']).size()
x= x.reset_index()
data = data.merge(x, left_on='monthDateYear', right_on='monthDateYear')
data = data.rename(columns={0: 'postsInThatDay'})

In [36]:
def seasonTime(date): 
    if pd.Timestamp('2019-6-14') > date >= pd.Timestamp('2019-4-13'): 
        return 'Playoffs' 
    elif pd.Timestamp('2019-4-13')> date > pd.Timestamp('2019-2-17'):
        return 'PostAllStarBreak'
    elif pd.Timestamp('2019-2-17')>= date >= pd.Timestamp('2019-2-15'):
        return 'AllStarBreak'
    elif pd.Timestamp('2019-2-15')> date >= pd.Timestamp('2018-10-16'):
        return 'BeforeAllStarBreak'
    elif pd.Timestamp('2018-6-8') > date >= pd.Timestamp('2018-4-14'): 
        return 'Playoffs' 
    elif pd.Timestamp('2018-4-14')> date > pd.Timestamp('2018-2-18'):
        return 'PostAllStarBreak'
    elif pd.Timestamp('2018-2-18')>= date >= pd.Timestamp('2018-2-16'):
        return 'AllStarBreak'
    elif pd.Timestamp('2018-2-16')> date >= pd.Timestamp('2017-10-17'):
        return 'BeforeAllStarBreak'
    elif pd.Timestamp('2017-6-12') > date >= pd.Timestamp('2017-4-15'): 
        return 'Playoffs' 
    elif pd.Timestamp('2017-4-15')> date > pd.Timestamp('2017-2-19'):
        return 'PostAllStarBreak'
    elif pd.Timestamp('2017-2-19')>= date >= pd.Timestamp('2017-2-17'):
        return 'AllStarBreak'
    elif pd.Timestamp('2017-2-17')> date >= pd.Timestamp('2016-10-25'):
        return 'BeforeAllStarBreak'
    else: return 'Offseason'

In [37]:
def vidAllStar(row): 
    if row['SeasonTime'] == 'AllStarBreak' and row['Type'] == 'Video': 
        return 1
    else:
        return 0 

In [38]:
def nonvidOffseason(row): 
    if row['SeasonTime'] == 'Offseason' and row['Type'] != 'Video': 
        return 1
    else:
        return 0 

In [39]:
data['Created']= pd.to_datetime(data['Created'])
data['SeasonTime'] = data.Created.map(lambda a: seasonTime(a))

In [40]:
data['vidAllStar'] = data.apply(lambda row: vidAllStar(row), axis =1 )
data['nonvidOffseason'] = data.apply(lambda row: nonvidOffseason(row), axis =1 )

In [41]:
data.dtypes

Engagements                    float64
Followers at Posting             int64
Created                 datetime64[ns]
Type                            object
Description                     object
Len_Desc                         int64
Tagged_Count                     int64
DayOfWeek                       object
playOffDate                      int64
month                            int64
hour                             int64
timeOfDay                       object
propOfSpecialTexts             float64
shoutouts                       object
teamTagged                       int64
otherTagged                      int64
top3Tagged                       int64
bottom3Tagged                    int64
hashtags                        object
hasHashtag                        bool
monthDateYear                   object
postsInThatDay                   int64
SeasonTime                      object
vidAllStar                       int64
nonvidOffseason                  int64
dtype: object

In [42]:
x= pd.get_dummies(data['Type'])
x['Not_Video'] = x['Album']+x['Photo']
data = data.join(x.drop(['Album','Photo'],axis = 1))
data = data.drop(['shoutouts','hashtags','monthDateYear','Created','Type','Description'],axis = 1)
data = data.join(pd.get_dummies(data['timeOfDay']))
data = data.join(pd.get_dummies(data['DayOfWeek']))
data = data.join(pd.get_dummies(data['month']))
data = data.join(pd.get_dummies(data['SeasonTime']))
data = data.drop(['DayOfWeek','timeOfDay','hour','month','SeasonTime'],axis = 1)

In [43]:
data.dtypes

Engagements             float64
Followers at Posting      int64
Len_Desc                  int64
Tagged_Count              int64
playOffDate               int64
propOfSpecialTexts      float64
teamTagged                int64
otherTagged               int64
top3Tagged                int64
bottom3Tagged             int64
hasHashtag                 bool
postsInThatDay            int64
vidAllStar                int64
nonvidOffseason           int64
Video                     uint8
Not_Video                 uint8
Afternoon                 uint8
Evening                   uint8
Morning                   uint8
Night                     uint8
Friday                    uint8
Monday                    uint8
Saturday                  uint8
Sunday                    uint8
Thursday                  uint8
Tuesday                   uint8
Wednesday                 uint8
1                         uint8
2                         uint8
3                         uint8
4                         uint8
5       

In [44]:
data= data.rename(columns={'Followers at Posting': 'NBA_Follower_Count'})

In [45]:
data.head()

,Engagements,NBA_Follower_Count,Len_Desc,Tagged_Count,playOffDate,propOfSpecialTexts,teamTagged,otherTagged,top3Tagged,bottom3Tagged,...,8,9,10,11,12,AllStarBreak,BeforeAllStarBreak,Offseason,Playoffs,PostAllStarBreak
0,NaN,36984682,55,1,1,0.111111,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
1,NaN,36984682,92,2,1,0.133333,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,NaN,36984682,60,3,1,0.444444,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,NaN,36955156,105,3,1,0.375000,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
4,NaN,36955156,121,3,1,0.375000,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [46]:
data['Tagged_Count^2'] = data['Tagged_Count']*data['Tagged_Count']

In [47]:
standardize = data[['NBA_Follower_Count', 'Len_Desc', 'Tagged_Count','Tagged_Count^2',
         'propOfSpecialTexts','postsInThatDay']]
standardize

,NBA_Follower_Count,Len_Desc,Tagged_Count,Tagged_Count^2,propOfSpecialTexts,postsInThatDay
0,36984682,55,1,1,0.111111,3
1,36984682,92,2,4,0.133333,3
2,36984682,60,3,9,0.444444,3
3,36955156,105,3,9,0.375000,6
4,36955156,121,3,9,0.375000,6
5,36955156,70,2,4,0.272727,6
6,36955156,34,0,0,0.166667,6
7,36955156,59,1,1,0.090909,6
8,36955156,80,2,4,0.133333,6
9,36928789,119,4,16,0.368421,1


In [48]:
dummy = data[['Engagements','teamTagged',
 'otherTagged',
 'top3Tagged',
 'bottom3Tagged',
 'hasHashtag',
 'vidAllStar',
 'nonvidOffseason',
 'Video',
 'Not_Video',
 'Afternoon',
 'Evening',
 'Morning',
 'Night',
 'Friday',
 'Monday',
 'Saturday',
 'Sunday',
 'Thursday',
 'Tuesday',
 'Wednesday',
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 'AllStarBreak',
 'BeforeAllStarBreak',
 'Offseason',
 'Playoffs',
 'PostAllStarBreak']]

In [49]:
len(dummy.columns.tolist())

38

In [52]:
X = preprocessing.scale(standardize)
Y = pd.DataFrame(X).rename(columns={0:'NBA_Follower_Count', 1:'Len_Desc', 2:'Tagged_Count',3:'Tagged_Count^2',
       4:'propOfSpecialTexts',5:'postsInThatDay'})
data_post_standardize= Y.join(dummy)

In [53]:
data_post_standardize['hasHashtag'] = data_post_standardize['hasHashtag'].astype(int)
data_post_standardize

,NBA_Follower_Count,Len_Desc,Tagged_Count,Tagged_Count^2,propOfSpecialTexts,postsInThatDay,Engagements,teamTagged,otherTagged,top3Tagged,...,8,9,10,11,12,AllStarBreak,BeforeAllStarBreak,Offseason,Playoffs,PostAllStarBreak
0,1.734250,-0.369289,-0.455505,-0.552255,-0.837799,0.007664,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1.734250,0.654361,0.472204,0.142405,-0.721902,0.007664,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,1.734250,-0.230958,1.399914,1.300174,0.900656,0.007664,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1.726429,1.014022,1.399914,1.300174,0.538478,1.923568,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
4,1.726429,1.456682,1.399914,1.300174,0.538478,1.923568,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,1.726429,0.045705,0.472204,0.142405,0.005089,1.923568,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
6,1.726429,-0.950279,-1.383215,-0.783809,-0.548056,1.923568,NaN,0,0,0,...,0,0,0,0,0,0,0,0,1,0
7,1.726429,-0.258624,-0.455505,-0.552255,-0.943159,1.923568,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
8,1.726429,0.322367,0.472204,0.142405,-0.721902,1.923568,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0
9,1.719446,1.401349,2.327624,2.921049,0.504167,-1.269606,NaN,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [54]:
data_post_standardize.to_csv('testing_data.csv', index= False)